#Deep Learning con Python
`Autor: Erwing FC 
~erwingforerocastro@gmail.com`

El siguiente ejercicio es un clasificador de texto segun el sentimiento del mismo (positivo o negativo) en base a comentarios de IMDB.

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
#descargamos los datos
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [ ]:
#contenido de los archivos
!ls aclImdb 
!ls aclImdb/test
!ls aclImdb/train
#contenido de un archivo de entrenamiento postivo
!cat aclImdb/train/pos/6248_7.txt

In [ ]:
#eliminamos los archivos no necesarios
!rm -r aclImdb/train/unsup

In [ ]:
# !pip install tf-nightly
#comando opcional para instalar las librerias de tf-nightly

In [ ]:
#creamos los dataset necesarios
batch_size=32 #tamaño de cada lote
raw_train_ds=tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",        #ruta
    batch_size=batch_size,  #tamaño del lote
    validation_split=0.2,   #20% del conjunto sera para validacion
    subset="training",      
    seed=1337
)
raw_val_ds=tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337
)
raw_test_ds=tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size,
)
print("Numero de lotes en raw_train_ds: %d" % tf.data.experimental.cardinality(raw_train_ds))
print("Numero de lotes en raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds))
print("Numero de lotes en raw_test_ds: %d" % tf.data.experimental.cardinality(raw_test_ds))

In [ ]:
#obervar el contenido de un ejemplo de entrenamiento
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

In [ ]:
#preparamos los datos
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re
#funcion para eliminar la etiqueta de salto html y la puntuación
def funcion_estandarizacion(input_data):
  minusculas=tf.strings.lower(input_data)
  eliminar_html=tf.strings.regex_replace(minusculas,"<br />"," ")
  return tf.strings.regex_replace(
      eliminar_html,"[%s]" % re.escape(string.punctuation),""
  )
#constantes del modelo
MAX_FEATURES=20000
EMBEDDING_DIM=128
SEQUENCE_LENGTH=500

vectorize_layer=TextVectorization(
    standardize=funcion_estandarizacion,
    max_tokens=MAX_FEATURES,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
)
#dataset de solo texto
text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [ ]:
#vectorizacion de los datos
#opcion 1 hacerlo parte del modelo

# text_input=tf.keras.Input(shape=(1,),dtype=tf.string,name="text")
# x=vectorize_layer(text_input)
# x=layers.Embedding(MAX_FEATURES + 1, EMBEDDING_DIM)(x)

#opcion 2 aplcarlo al conjunto de datos de texto
def vectorizacion_texto(text,label):
  text=tf.expand_dims(text,-1)
  return vectorize(text),label

#vectorizamos los datos
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Mejoramos el desempeño al obtener los datos de forma asincrona
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [ ]:
#construimos el modelo 
from tensorflow.keras import layers
#un valor entero como entrada (los indices del vocabulario)
inputs =tf.keras.Input(shape=(None,),dtype="int64")

#luego mapeamos los indices en un espacio dimensional
x=layers.Embedding(MAX_FEATURES,EMBEDDING_DIM)(inputs)
x=layers.Dropout(0.5)(x)

x=layers.Conv1D(128,7,padding="valid",activation="relu",strides=3)(x)
x=layers.Conv1D(128,7,padding="valid",activation="relu",strides=3)(x)
x=layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epochs = 3
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
#evaluación del modelo
model.evaluate(test_ds)

In [ ]:
# Una entrada de texto
inputs = tf.keras.Input(shape=(1,), dtype="string")
# convercion de texto a indices del vocabulario
indices = vectorize_layer(inputs)
# convertir los indices del vocabulario en predicciones
outputs = model(indices)

# Fin del modelo
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# probamos con el test sin procesar 
end_to_end_model.evaluate(raw_test_ds)